This file is using POD to deal with FPC data

## Mounting your google drive

You can use google drive to store and access files e.g. storing and loading data from numpy or CSV files.  
Use the following command to mount your GDrive and access your files.

In [18]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [19]:
!pip install ffmpeg
!pip install vtk

In [20]:
import os
# change the current path. The user can adjust the path depend on the requirement
os.chdir("/content/gdrive/MyDrive/Cola-Notebooks/FYP/YF")
import vtktools

In [21]:
# !unzip csv_data.zip 

In [22]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import numpy.linalg as la
import scipy.linalg as sl
import scipy.sparse.linalg as spl
import matplotlib.pyplot as plt
import torch.nn as nn  # Neural network module
import scipy.sparse as sp
import scipy.optimize as sop
import progressbar
# making slopes
import torch
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from matplotlib.pyplot import LinearLocator
import matplotlib as mpl
import matplotlib.colors as colors


# create an animation
from matplotlib import animation
from IPython.display import HTML

from matplotlib import animation
import math
import ffmpeg

!pip install pycm livelossplot
%pylab inline
from livelossplot import PlotLosses

from torch.utils.data import DataLoader
import torch.utils.data as Data

import time
import platform
print('python version', platform.python_version())
print('torch version', torch.__version__)
print('numpy version', np.version.version)

Populating the interactive namespace from numpy and matplotlib
python version 3.7.11
torch version 1.9.0+cu102
numpy version 1.19.5


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['ma']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [23]:
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True

device = 'cuda'  # Set out device to GPU

print('Cuda installed, running on GPU!')  # print sentence

Cuda installed, running on GPU!


In [24]:
# These functions are saved in function.py and the note are also added to that file
def saveIndex(path_train, path_valid, path_test,train_index, valid_index, test_index):
    # save training and validation loss    
    np.savetxt(path_train,train_index, delimiter=',')
    np.savetxt(path_valid,valid_index, delimiter=',')
    np.savetxt(path_test,test_index, delimiter=',')

def getIndex(path_train,path_valid,path_test):
    train_index = np.loadtxt(path_train,delimiter=",")
    valid_index = np.loadtxt(path_valid,delimiter=",")
    test_index = np.loadtxt(path_test,delimiter=",")
    return train_index,valid_index,test_index


# SVD

## Load data

In [25]:
os.chdir('/content/gdrive/MyDrive/Cola-Notebooks/FYP/YF')
print(os.getcwd())

# read in the data (1000 csv files)
nTrain = 1600
nValid = 200
nTest = 200
nTotal = nTrain + nValid + nTest
nNodes = 20550 # should really work this out


# The below method to load data is too slow. Therefore, we use load pt file

# [:, :, 2] is speed, [:, :, 3] is u, [:, :, 4] is v
# (speed not really needed)
# [:, :, 0] and [:, :, 1] are the SFC orderings

# training_data = np.zeros((nTrain,nNodes,5))
# for i in range(nTrain):
#     data = np.loadtxt('csv_data/data_' +str(int(train_index[i]))+ '.csv', delimiter=',')
#     training_data[i,:,:] = data
# training_data = np.array(training_data)
# print('size training data', training_data.shape)

# valid_data = np.zeros((nValid,nNodes,5))
# for i in range(nValid):
#     data = np.loadtxt('csv_data/data_' +str(int(valid_index[i]))+ '.csv', delimiter=',')
#     valid_data[i,:,:] = data
# valid_data = np.array(valid_data)
# print('size validation data', valid_data.shape)

# test_data = np.zeros((nTest,nNodes,5))
# for i in range(nTest):
#     data = np.loadtxt('csv_data/data_' +str(int(test_index[i]))+ '.csv', delimiter=',')
#     test_data[i,:,:] = data
# test_data = np.array(test_data)
# print('size test data', test_data.shape)

# total_data = np.zeros((nTotal,nNodes,5))
# for i in range(len(train_index)):
#     total_data[int(train_index[i]),:,:] = training_data[i,:,:]

# for i in range(len(valid_index)):
#     total_data[int(valid_index[i]),:,:] = valid_data[i,:,:]

# for i in range(len(test_index)):
#     total_data[int(test_index[i]),:,:] = test_data[i,:,:]
# print('size total data', total_data.shape)



/content/gdrive/MyDrive/Cola-Notebooks/FYP/YF


In [26]:
# Before we save the pt file, we must load the data according to the above method
# torch.save(training_data, '/content/gdrive/MyDrive/FPC_new_random_train.pt')
# torch.save(valid_data, '/content/gdrive/MyDrive/FPC_new_random_valid.pt')
# torch.save(test_data, '/content/gdrive/MyDrive/FPC_new_random_test.pt')
# torch.save(total_data, '/content/gdrive/MyDrive/FPC_new_random_total.pt')


In [27]:
# load the data, this method save the time
training_data = torch.load('/content/gdrive/MyDrive/FPC_new_random_train.pt')
valid_data = torch.load('/content/gdrive/MyDrive/FPC_new_random_valid.pt')
test_data = torch.load('/content/gdrive/MyDrive/FPC_new_random_test.pt')
total_data = torch.load('/content/gdrive/MyDrive/FPC_new_random_total.pt')
print(training_data.shape)
print(valid_data.shape)
print(test_data.shape)
print(total_data.shape)

(1600, 20550, 5)
(200, 20550, 5)
(200, 20550, 5)
(2000, 20550, 5)


In [28]:
# Load the index of different part data
total_index = []
path_train = "/content/gdrive/MyDrive/Cola-Notebooks/FYP/YF/"+"new_FPC_train_index.csv"
path_valid = "/content/gdrive/MyDrive/Cola-Notebooks/FYP/YF/"+"new_FPC_valid_index.csv"
path_test = "/content/gdrive/MyDrive/Cola-Notebooks/FYP/YF/"+"new_FPC_test_index.csv"
# saveIndex(path_train, path_valid, path_test,train_index, valid_index, test_index)

train_index,valid_index,test_index= getIndex(path_train,path_valid,path_test)
print(test_index)
for i in range(2000):
    total_index.append(i)

[ 133.  490. 1480.  730.  481. 1382.  440.  750. 1502. 1451.  692. 1094.
 1679.  510. 1241. 1101.  543. 1312. 1432. 1988. 1148. 1801. 1519.  367.
 1858. 1043. 1175. 1218. 1479.  103. 1363.  800.  258. 1851.  267.  999.
  611. 1824.  318.  753. 1413.  727. 1273. 1358. 1090.  838.  250. 1763.
 1038.  439. 1199.  334. 1848. 1924. 1013.  271.  936.  600. 1553.  423.
 1467. 1658.  929. 1748.  783.  329.  303. 1067.  868.  374. 1102. 1843.
  683.  449.  855. 1142. 1393.  194. 1112.  636. 1617. 1910. 1722.  536.
 1149. 1765.  468. 1922. 1703. 1311.  341.  110. 1258. 1257. 1711.   93.
 1969.  396. 1259.  199.  962. 1704.  462. 1407.  634.  535. 1505.  537.
  612. 1707. 1565. 1963. 1955.    3. 1058. 1946.  372. 1653. 1077.  414.
  469.  680. 1430.  649.  215.  234. 1692.  653. 1455.  582. 1169. 1138.
  411.  518.  865. 1977. 1688.  822.  397. 1388. 1221.  239.  249. 1781.
 1751.  915.  278. 1970.  907.  477. 1552.  703.  870.  916. 1650.  561.
 1401.  129. 1123. 1804. 1871. 1527.  308.   94. 19

In [29]:
# rescale the data so that u and v data lies in the range [-1,1] (and speed in [0,1])
ma = np.max(training_data[:, :, 2])
mi = np.min(training_data[:, :, 2])
k = 1./(ma - mi)
b = 1 - k*ma #k*mi
training_data[:, :, 2] = k * training_data[:, :, 2] + b #- b
# this won't be used

ma = np.max(training_data[:, :, 3])
mi = np.min(training_data[:, :, 3])
ku = 2./(ma - mi)
bu = 1 - ku*ma 
training_data[:, :, 3] = ku * training_data[:, :, 3] + bu
valid_data[:, :, 3] = ku * valid_data[:, :, 3] + bu
test_data[:, :, 3] = ku * test_data[:, :, 3] + bu
total_data[:, :, 3] = ku * total_data[:, :, 3] + bu

ma = np.max(training_data[:, :, 4])
mi = np.min(training_data[:, :, 4])
kv = 2./(ma - mi)
bv = 1 - kv*ma
training_data[:, :, 4] = kv * training_data[:, :, 4] + bv
valid_data[:, :, 4] = kv * valid_data[:, :, 4] + bv
test_data[:, :, 4] = kv * test_data[:, :, 4] + bv
total_data[:, :, 4] = kv * total_data[:, :, 4] + bv

In [30]:
# Because it has two parameters. Reshapr the data from(2000,20550,2) to (2000,20550*2)
snaps = np.zeros([nTrain,nNodes])
snaps1 = np.zeros([nTrain,nNodes])
snaps2 = np.zeros([nValid,nNodes])
snaps3 = np.zeros([nValid,nNodes])
snaps4 = np.zeros([nTest,nNodes])
snaps5 = np.zeros([nTest,nNodes])

for i in range(nTrain):
    snaps[i,:] = training_data[i,:,3]
    snaps1[i,:] = training_data[i,:,4]

snapstrain = np.c_[snaps,snaps1]


for i in range(nValid):
    snaps2[i,:] = valid_data[i,:,3]
    snaps3[i,:] = valid_data[i,:,4]

snapsvalid = np.c_[snaps2,snaps3]


for i in range(nTest):
    snaps4[i,:] = test_data[i,:,3]
    snaps5[i,:] = test_data[i,:,4]

snapstest = np.c_[snaps4,snaps5]

snapstotal = np.zeros([total_data.shape[0],20550*2])
for i in range(nTrain+nValid+nTest):
    snapstotal[i,:nNodes] = total_data[i,:,3]
    snapstotal[i,nNodes:] = total_data[i,:,4]

print('snapstrain shape: ',snapstrain.shape)
print('snapsvalid shape: ',snapsvalid.shape)
print('snapstest shape: ',snapstest.shape)
print('snapstotal shape: ',snapstotal.shape)

snapstrain shape:  (1600, 41100)
snapsvalid shape:  (200, 41100)
snapstest shape:  (200, 41100)
snapstotal shape:  (2000, 41100)


In [31]:
# The solution is usually arranged in a column vector
snapstrain = snapstrain.transpose()
snapsvalid = snapsvalid.transpose()
snapstest = snapstest.transpose()
snapstotal = snapstotal.transpose()
print('snapetrain shape',snapstrain.shape)
print('snapevalid shape',snapsvalid.shape)
print('snapetest shape',snapstest.shape)
print('snapetotal shape',snapstotal.shape)

snapetrain shape (41100, 1600)
snapevalid shape (41100, 200)
snapetest shape (41100, 200)
snapetotal shape (41100, 2000)


## SVD of the snapshot matrix

In [32]:
# Using la.svd to do the singular value decomposition
t_train_0 = time.time()
u,sigma2,vt = la.svd(snapstrain,False)
t_train_1 = time.time()
print('u shape:',u.shape)
print('sigma2 shape:',sigma2.shape)
print('vt shape: ',vt.shape)
print("Train time: ",t_train_1-t_train_0)

u shape: (41100, 1600)
sigma2 shape: (1600,)
vt shape:  (1600, 1600)
Train time:  14.94536280632019


MSE equation
$$\frac1N{\textstyle\sum_{p=1}^N}{\textstyle(}{\textstyle{\scriptstyle u}^{t_p}}{\textstyle-}{\textstyle R}{\textstyle{\scriptstyle R}^T}{\textstyle{\scriptstyle u}^{t_p}}{\textstyle)}{\textstyle\cdot}{\textstyle(u^{t_p}-RR^Tu^{t_p})}$$

The relative square error
$$\frac{\frac1N\sum_{p=1}^N(u^{t_p}-RR^Tu^{t_p})\cdot(u^{t_p}-RR^Tu^{t_p})}{\frac1N\sum_{p=1}^N(u^{t_p})\cdot(u^{t_p})}$$

In [33]:
# The function is used to calculate the mean square error accoridng to the above equation
def MSE(S,index, R, RT):
    m = zeros([2*nNodes,len(index)])
    n = zeros([len(index),nNodes,2])
    for i in range(len(index)):
        middle = RT@S[:,int(index[i])]
        m[:,i] = S[:,int(index[i])] - R @ middle
        n[i,:,0] = m[:nNodes,i]
        n[i,:,1] = m[nNodes:,i]
    return (n**2).mean()


# def Error(S, index, R, RT):
#     m = zeros([2*nNodes,len(index)])
#     n = zeros([len(index),nNodes,2])
#     error = zeros([len(index),i])
#     for i in range(len(index)):
#         middle = RT@S[:,int(index[i])]
#         m[:,i] = S[:,int(index[i])] - R @ middle
#         n[i,:,0] = m[:nNodes,i]
#         n[i,:,1] = m[nNodes:,i]
#         error[i,0] = (n[i,:,:]**2).mean()
#     return error

# THis function is used to calculate the relative square error according to the above equation
def relative_error(S, index, R, Rt):
    m = zeros([2*nNodes,len(index)])
    n = zeros([len(index),nNodes,2])
    h = zeros([len(index),nNodes,2])
    for i in range(len(index)):
        middle = RT@S[:,int(index[i])]
        m[:,i] = S[:,int(index[i])] - R @ middle
        h[i,:,0] = S[nNodes:,int(index[i])]
        h[i,:,1] = S[:nNodes,int(index[i])]
        n[i,:,0] = m[:nNodes,i]
        n[i,:,1] = m[nNodes:,i]
    return (n**2).mean()/(h**2).mean()

# This function is used to reconstruct data from the reduced variable
def orginal(S,R,RT):
    N = len(S[1])
    # print('N shape',N)
    orginal = np.zeros([2*nNodes,N])
    for i in range(N):
        middle = RT@S[:,i]
        orginal[:,i] = R@middle
    return orginal

### 1 variable

In [34]:
# Choose the number of basis function
R = u[:,0:1]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 1)
RT shape: (1, 41100)


In [35]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)
print(result.shape)

Reconstruction: 3.2953226566314697
(41100, 2000)


In [36]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  0.021071473294830976
The valid MSE:  0.02082311044842053
The test MSE:  0.02129882757690237
The total MSE:  0.0210693724383971
The train relative square error:  0.2930264195339251
The valid relative square error:  0.2906072995373513
The test relative square error:  0.2948952056775621
The total relative square error:  0.29297307116573224


### 2 variable

In [37]:
# Choose the number of basis function
R = u[:,0:2]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 2)
RT shape: (2, 41100)


In [38]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 2.25429105758667


In [40]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  0.01153585065129064
The valid MSE:  0.010870498227003558
The test MSE:  0.0123944830594379
The total MSE:  0.011555178649676633
The train relative square error:  0.16042110417855854
The valid relative square error:  0.15170865765708355
The test relative square error:  0.17160914692993542
The total relative square error:  0.16067664980352925


### 4 variable

In [41]:
# Choose the number of basis function
R = u[:,0:4]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 4)
RT shape: (4, 41100)


In [42]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 2.4365694522857666


In [43]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  0.002778288171449406
The valid MSE:  0.0027700291476035767
The test MSE:  0.0025904994995278835
The total MSE:  0.00275868340187267
The train relative square error:  0.038635733910119346
The valid relative square error:  0.0386585228090113
The test relative square error:  0.03586703915803049
The total relative square error:  0.03835994408393747


### 8 variable

In [44]:
# Choose the number of basis function
R = u[:,0:8]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 8)
RT shape: (8, 41100)


In [45]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 2.345869302749634


In [46]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  0.0016314512027842728
The valid MSE:  0.001567985800200461
The test MSE:  0.0014226494733828626
The total MSE:  0.0016042244895857491
The train relative square error:  0.02268746460711236
The valid relative square error:  0.021882807577564962
The test relative square error:  0.01969744614089834
The total relative square error:  0.02230700401387807


### 16 variable

In [47]:
# Choose the number of basis function
R = u[:,0:16]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 16)
RT shape: (16, 41100)


In [48]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 3.8966381549835205


In [49]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  0.0007154732169125436
The valid MSE:  0.0006605267440753894
The test MSE:  0.0006522750422584215
The total MSE:  0.0007036587521634185
The train relative square error:  0.009949591663138793
The valid relative square error:  0.009218310292471612
The test relative square error:  0.009031144181557475
The total relative square error:  0.009784490082783268


### 32 variable

In [50]:
# Choose the number of basis function
R = u[:,0:32]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 32)
RT shape: (32, 41100)


In [51]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 4.052673816680908


In [52]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  0.00026879591002737074
The valid MSE:  0.0002628113410994816
The test MSE:  0.0002591130344208928
The total MSE:  0.000267229165573934
The train relative square error:  0.0037379589931191533
The valid relative square error:  0.0036677946992546085
The test relative square error:  0.0035875773585843265
The total relative square error:  0.0037158652718375686


### 64 variable

In [53]:
# Choose the number of basis function
R = u[:,0:64]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 64)
RT shape: (64, 41100)


In [54]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 5.38860559463501


In [55]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  7.060790934629863e-05
The valid MSE:  8.062858622131723e-05
The test MSE:  7.631131863237528e-05
The total MSE:  7.218031796240788e-05
The train relative square error:  0.000981895407930365
The valid relative square error:  0.001125252433604068
The test relative square error:  0.0010565765614265407
The total relative square error:  0.0010036791315448646


#### Convert csv to vtu

Convet csv to vtu so that we can watch them using paraview

In [ ]:
print('result shape: ',result.shape)
result_recover = result.transpose()
print('results_recover shape:',result_recover.shape)
# Splite the velocity u and v, convert shape from (2000,41100) to (2000,20550,2)
rec_results = np.zeros([2000,20550,2])
for i in range(2000):
    rec_results[i,:,0] = result_recover[i,:nNodes]
    rec_results[i,:,1] = result_recover[i,nNodes:]
print("rec_results shape: ",rec_results.shape)

result shape:  (41100, 2000)
results_recover shape: (2000, 41100)
rec_results shape:  (2000, 20550, 2)


In [ ]:
# Before convert csv file to vtu file, the range of data must be recovered
rec_results[:,:,0] = (rec_results[:,:,0] - bu)/ku
rec_results[:,:,1] = (rec_results[:,:,1] - bv)/kv
total_data[:, :, 3] = (total_data[:, :, 3] - bu)/ku
total_data[:, :, 4] = (total_data[:, :, 4] - bv)/kv

In [ ]:
results = rec_results
print('results shape', results.shape)
N = results.shape[1] * results.shape[2]
results = results.reshape((results.shape[0],N), order='F')
print('results shape', results.shape, type(results))
# The path can be defined by user depending on the requirements
path = "./HAE/All_results/SVD"+"_LV"+str(64)+"_result.csv"
## write results to file
np.savetxt(path, results , delimiter=',')

results shape (2000, 20550, 2)
results shape (2000, 41100) <class 'numpy.ndarray'>


In [ ]:
# convert the error data to csv 
error = rec_results - total_data[:,:,3:5]
results = error
print('results shape', results.shape)
N = results.shape[1] * results.shape[2]
results = results.reshape((results.shape[0],N), order='F')
print('results shape', results.shape, type(results))
# The path can be defined by user depending on the requirements
path = "./HAE/All_results/SVD"+"_LV"+str(64)+"_error.csv"
## write results to file
np.savetxt(path, results , delimiter=',')

results shape (2000, 20550, 2)
results shape (2000, 41100) <class 'numpy.ndarray'>


### 128 variable

In [56]:
# Choose the number of basis function
R = u[:,0:128]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 128)
RT shape: (128, 41100)


In [57]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 9.729057788848877


In [58]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  1.2090126037297648e-05
The valid MSE:  1.7100131500400123e-05
The test MSE:  1.6411809948911704e-05
The total MSE:  1.3023294974769332e-05
The train relative square error:  0.00016812902898879288
The valid relative square error:  0.00023864941068118934
The test relative square error:  0.00022723147802153608
The total relative square error:  0.00018109104751986096


### 256 variable

In [59]:
# Choose the number of basis function
R = u[:,0:256]
print('R shape:', R.shape)
RT = R.transpose()
print('RT shape:',RT.shape)

R shape: (41100, 256)
RT shape: (256, 41100)


In [60]:
t_pre_0 = time.time()
result = orginal(snapstotal,R,RT)
t_pre_1 = time.time()
print('Reconstruction:',t_pre_1-t_pre_0)

Reconstruction: 18.19143581390381


In [61]:
print("The train MSE: ",MSE(snapstotal, train_index, R, RT))
print("The valid MSE: ",MSE(snapstotal, valid_index, R, RT))
print("The test MSE: ",MSE(snapstotal, test_index, R, RT))
print("The total MSE: ",MSE(snapstotal, total_index, R, RT))
print("The train relative square error: ",relative_error(snapstotal, train_index, R, RT))
print("The valid relative square error: ",relative_error(snapstotal, valid_index, R, RT))
print("The test relative square error: ",relative_error(snapstotal, test_index, R, RT))
print("The total relative square error: ",relative_error(snapstotal, total_index, R, RT))

The train MSE:  1.5128647299761146e-06
The valid MSE:  3.39044289008276e-06
The test MSE:  3.22734056735203e-06
The total MSE:  1.8720701297243711e-06
The train relative square error:  2.103836446845922e-05
The valid relative square error:  4.731701610876773e-05
The test relative square error:  4.468449058824828e-05
The total relative square error:  2.6031441465406337e-05
